In [154]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests

In [155]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(30) 
    return soup

In [156]:
def get_data_hubpages_web(soup):
    all_articles = soup.find_all('section',attrs={'class':'m-card-group'})
    # Declaration
    articles_topic = []
    articles_content = []
        
    for group_of_article in all_articles:
        
        articles = group_of_article.find_all('div',attrs={'class':'l-grid--item'})
        for article in articles:
            
            if article.find('h2') != None:
                topic = article.find('h2').get_text()
                articles_topic.append(topic)
                
                url_of_article = article.find('a')['href']
                soup = get_article_html("https://discover.hubpages.com/" + url_of_article)

                content = soup.find('div',attrs={'class':'m-detail--body'}).find_all("p")
                full_content = ""
                if content != None:
                    for paragraph in content:
                        full_content = full_content + " " + paragraph.get_text().strip()
                articles_content.append(full_content) 
      
    print("Finish get data!")     
    return articles_topic, articles_content

In [157]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'hubpages Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["politics", "sports", "technology", "business", "art", "entertainment", "health", "travel", "religion-philosophy",
              "money", "community", "home", "food", "style", "education", "literature", "autos", "games-hobbies",
              "relationships", "holidays"]
print("Start scraping from hubpages!")
for category in categories:
    soup = get_article_html(f"https://discover.hubpages.com/{category}")
    topic, content = get_data_hubpages_web(soup)
    insert_data_to_csv(topic, category, content)
print("Finish scraping from hubpages.com!")

Start scraping from hubpages!
